In [1]:
import torch

In [2]:
torch.__version__

'2.3.1+cu121'

In [3]:
import transformers

In [4]:
transformers.__version__

'4.42.4'

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
sentence = "I skip across the"

In [8]:
input_ids = tokenizer(sentence, return_tensors='pt').input_ids

In [9]:
input_ids

tensor([[   40, 14267,  1973,   262]])

In [10]:
# words --> tokens --> Unique ID --> vector embed

In [11]:
for token_id in input_ids[0]:
  print(tokenizer.decode(token_id))

I
 skip
 across
 the


In [12]:
from transformers import AutoModelForCausalLM

In [13]:
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
outputs = gpt2(input_ids)

In [15]:
outputs.logits.shape

torch.Size([1, 4, 50257])

In [16]:
final_logits = gpt2(input_ids).logits[0, -1]

In [17]:
final_logits

tensor([-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209],
       grad_fn=<SelectBackward0>)

In [18]:
final_logits.argmax() #Token ID --> Index Location Logits

tensor(1627)

In [19]:
tokenizer.decode(final_logits.argmax())

' line'

In [20]:
top10_logits = torch.topk(final_logits, 10)

In [21]:
for index in top10_logits.indices:
  print(tokenizer.decode(index))

 line
 street
 river
 room
 pond
 bridge
 border
 country
 road
 board


In [22]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)

In [23]:
for value, index in zip(top10.values, top10.indices):
  print(f"{tokenizer.decode(index)} -- {value.item(): .1%}") #Chance for another word being after 'the' in sentence

 line --  3.6%
 street --  2.7%
 river --  2.2%
 room --  1.9%
 pond --  1.8%
 bridge --  1.7%
 border --  1.7%
 country --  1.7%
 road --  1.6%
 board --  0.9%


In [33]:
#Generating Text
output_ids = gpt2.generate(input_ids, max_new_tokens=20, do_sample=True, temperature=1.3)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
decoded_text = tokenizer.decode(output_ids[0])

In [35]:
print(decoded_text)

I skip across the main road and you see no sign of it, because there is no sign of anything here you're


In [37]:
#Text Generation
from transformers import pipeline

In [38]:
pipe = pipeline('text-generation', model='openai-community/gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [39]:
pipe("I went to the happy store today and bought a")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I went to the happy store today and bought a gift certificate for $30. It read, "What is the Secretariat here?" It was a gift to a young girl I knew as "Sharon", or just a sweet person, who would'}]